<a href="https://colab.research.google.com/github/mouhsineguet/Data-Science-portfolio/blob/main/NLP_Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##In this Notebook we will aplly NLP techniques on PeerJ article

### Importing the neccessary packages and the texts file for testing





In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:

!pip install transformers
#!pip install sentencepiece
!pip install -q 'spacy[cuda-autodetect,transformers,lookups]'

In [ ]:
!pip install gensim==3.7.0

In [ ]:
import pandas as pd
import numpy as np
import gensim
import spacy

# import TensorFlow
import tensorflow as tf

#Check the version of TensorFlow you are using
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


2.12.0
[]


In [4]:
with open('/content/Introduction.txt',encoding='utf-8') as f:
  intro = f.read()

with open('/content/Materials.txt',encoding='utf-8') as f:
  materials = f.read()
  
with open('/content/abstraction.txt',encoding='utf-8') as f:
  abstraction = f.read()

with open('/content/DISCUTION.txt',encoding='utf-8') as f:
  discution = f.read()

with open('/content/bascketball.txt',encoding='utf-8') as f:
  bascketball = f.read()

with open('/content/endometroise.txt',encoding='utf-8') as f:
  endemetriose = f.read()

with open('/content/infertility.txt',encoding='utf-8') as f:
  infertility = f.read()

## Summarazation

### Extractive method: using PageRank algo in Gensim

In [ ]:
def textrank(corpus, ratio=0.2):    
    if type(corpus) is str:        
       corpus = [corpus]    
    lst_summaries = [gensim.summarization.summarize(txt,  
                     ratio=ratio) for txt in corpus]    
    return lst_summaries

## Apply the function to corpus
predicted = textrank(corpus=intro, ratio=0.2)
predicted

NameError: ignored

In [ ]:
#for abstract 
textrank(corpus=abstraction, ratio=0.2)

['Methods: We performed small RNA sequencing in 20 patients with HCC and 19\nEleven patients with HCC had chronic liver disease, and analyses were\npublished report of miRNAs in liver tissue of patients with HCC vs cirrhosis.\ndetermine if salivary miRNAs have predictive potential for detecting HCC.\nand in a previously published study of miRNAs in HCC tissue compared to cirrhotic\nHCC are detectable in saliva, that salivary miRNA signatures show potential to be']

In [ ]:
#for materials 
textrank(corpus=materials, ratio=0.2)

['Patient recruitment and sample collection\nSaliva samples were collected from 20 individuals with HCC and 19 from individuals with\nSome patients with HCC had previously received treatment\nfor patients with HCC can be found in Table S1.\nTwelve out of the twenty patients with HCC had CLD, and analyses were\nmiRNAs followed by cDNA synthesis with UMI assignment, cDNA cleanup, library\nprocessed using the miRNAeasy Serum/Plasma Kit. Adapter dilutions throughout the\nprotocol followed the serum/plasma recommendations.\nfollowed that of a 10 ng input sample with 19 total cycles, consistent with manufacturer\nGermany) with an input of 10 ng RNA were processed alongside the saliva samples.']

In [ ]:
#for discution 
textrank(corpus=discution, ratio=0.2)

['Current approaches used to screen patients for HCC lack sensitivity, resulting in too many\nMaking screening tools more widely accessible may result in detection of HCC prior to the\nSalivary miRNAs have demonstrated biomarker potential for CLD and cancers, such\nHere, we observed that a large fraction of the miRNAs detected were differentially\nexpressed in HCC patients vs cirrhosis (365 out of 4,565 detected miRNA), and the\nmajority of these (78%, N = 283) were significantly down-regulated in patients with HCC\nImportantly, in patients with both CLD and HCC, 248\nmiRNAs were significantly down-regulated compared to patients with cirrhosis without\nclassified the majority of HCC samples correctly (Table 4) suggesting that the miRNA']

### Abstractive method using Transformers


#### 1_BERT

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
import transformers
summarizer = transformers.pipeline("summarization")   
def summary(corpus,summarizer):
  sum=[]
  for text in corpus:
    summary = summarizer(text, max_length=100, min_length=10, do_sample=False)
    sum.append(summary[0]['summary_text'])
  return sum



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
## Apply the function to corpus
corpus=[intro,abstraction,materials,discution]
paragraphs=['introduction','abstraction','materials','discution']
for para,summaries in zip(paragraphs,summary(corpus,summarizer)):
  print(para)
  print(summaries)

introduction
 Liver cancer is the most rapidly increasing cancer among men and women in the United States and is estimated to have resulted in 31,780 deaths in 2019 . Hepatocellular carcinoma (HCC) accounts for 80% of all primary liver cancers, and the global incidence of HCC is expected to increase to 78 million by 2030 .
abstraction
 MicroRNAs (miRNAs) are small, non-coding RNAs that regulate many cellular functions and impact cancer development and progression . MiRNAs are detectable in saliva and have shown potential as non-invasivebiomarkers for a number of cancers including breast, oral, and lung cancers .
materials
 Saliva samples were collected from 20 individuals with HCC and 19 from individuals with liver disease seen at the Cleveland Clinic (Cleveland, OH) Participants were adult patients who underwent liver transplantation for HCC, surgical resection for liver tumors or liver biopsy . All participants provided written informed consent and study was approved by Cleveland Cli

### Test sur un text en français


In [ ]:
import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
tokenizer = RobertaTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)
def generate_summary(text):
   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)
   



The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

generate_summary(justice)


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'La justice est un principe philosophique, juridique et moral fondamental : suivant ce principe, les actions humaines doivent être approuvées ou rejetées en fonction de leur mérite au regard de la morale (le bien, le droit, la vertu ou de toute autre norme de jugement des comportements).'

#### 2_T5 by Google

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('t5-small', n)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def generate_summary(text):
    # Clean the text
    text = text.strip()

    # Tokenize the text and add special tokens for T5
    input_ids = tokenizer.encode('summarize: ' + text, return_tensors='pt')

    # Generate the summary
    summary_ids = model.generate(input_ids)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, new)

    return summary


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-3b')

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-3b')

# Define the input text
input_text = """ Liver cancer is the most rapidly increasing cancer among men and women in the United
States and is estimated to have resulted in 31,780 deaths in 2019 (American Cancer Society,
2019). Hepatocellular carcinoma (HCC) accounts for 80% of all primary liver cancers,
and the global incidence of HCC is expected to increase to 78 million by 2030 (Petrick
et al., 2016) due to increasing nonalcoholic steatohepatitis (NASH), hepatitis C, and
excessive alcohol consumption. Early detection of HCC has shown to improve reception of
curative therapy and overall survival (Singal, Pillai & Tiro, 2014). However, current
HCC serum biomarkers, such as alpha fetoprotein (AFP) lack prognostic and diagnostic
value (39–65% sensitivity) (Daniele et al., 2004). Although resection is the treatment of
choice for many patients, as many as 54% will recur within 2 years, and ~70% within
5 years (Forner et al., 2006; Tabrizian et al., 2015). As early detection is associated with
improved prognosis and overall survival, reliable biomarkers are needed to detect tumor in
early stages of development.
MicroRNAs (miRNAs) are small, non-coding RNA molecules that regulate gene
expression by translational inhibition or mRNA degradation. Although the majority of
miRNA regulation occurs intracellularly, in some cases select miRNAs are found in the
circulation and have been associated with a plethora of human diseases. Several studies
have shown miRNA expression is associated with cancer (Eslam, Valenti & Romeo, 2018),
NASH/NAFLD and alcoholic hepatitis (Blaya et al., 2016; Eslam, Valenti & Romeo,
2018), and patient survival (Pineau et al., 2010; Dongiovanni et al., 2018). In addition,
differentially expressed miRNA in blood have been detected in HCC (Murakami et al.,
2012; Qi et al., 2013). However, it is not known if detectable alterations in salivary miRNAs
are present in patients with HCC. Previously, miRNA abundance in saliva has been
associated with oral, head and neck, lung, and breast cancers (Zhang et al., 2010, 2012;
Elashoff et al., 2012; Salazar, Calvopiña & Punyadeera, 2014; Salazar et al., 2014; Wan
et al., 2017). Saliva is an attractive biospecimen for biomarkers because (1) it is noninvasive, (2) less expensive than more invasive biospecimens, (3) can be collected without
requiring a patient to go to the doctor’s office, and (4) saliva collection tubes can now
stabilize samples at room temperature for months (Nunes et al., 2012; Lim et al., 2016),
allowing """

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate the summary
summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Summary:", summary)


In [ ]:
## Apply the function to corpus
text=""" Liver cancer is the most rapidly increasing cancer among men and women in the United
States and is estimated to have resulted in 31,780 deaths in 2019 (American Cancer Society,
2019). Hepatocellular carcinoma (HCC) accounts for 80% of all primary liver cancers,
and the global incidence of HCC is expected to increase to 78 million by 2030 (Petrick
et al., 2016) due to increasing nonalcoholic steatohepatitis (NASH), hepatitis C, and
excessive alcohol consumption. Early detection of HCC has shown to improve reception of
curative therapy and overall survival (Singal, Pillai & Tiro, 2014). However, current
HCC serum biomarkers, such as alpha fetoprotein (AFP) lack prognostic and diagnostic
value (39–65% sensitivity) (Daniele et al., 2004). Although resection is the treatment of
choice for many patients, as many as 54% will recur within 2 years, and ~70% within
5 years (Forner et al., 2006; Tabrizian et al., 2015). As early detection is associated with
improved prognosis and overall survival, reliable biomarkers are needed to detect tumor in
early stages of development. """

print(generate_summary(text))


liver cancer is the most rapidly increasing cancer among men and women in the united states.


## Topics Modeling, Text Categorization:

---




### Unsupervised method using Spacy(Clustring)

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


def preprocess(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub('\d+', '', text)
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join the tokens back into a string
    text = ' '.join(tokens)
    
    return text


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string
from collections import Counter
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

nlp = spacy.load('en_core_web_sm')
parser = English()

# Sample corpus of texts
texts = [
    "Liver cancer is the most rapidly increasing cancer among men and women in the United States and is estimated to have resulted in 31,780 deaths in 2019.",
    "Hepatocellular carcinoma (HCC) accounts for 80% of all primary liver cancers, and the global incidence of HCC is expected to increase to 78 million by 2030.",
    "Early detection of HCC has shown to improve reception of curative therapy and overall survival.",
    "Several studies have shown miRNA expression is associated with cancer and patient survival.",
    "Saliva is an attractive biospecimen for biomarkers because it is non-invasive and can be collected without requiring a patient to go to the doctor’s office."
]

# Preprocessing function
"""def preprocess(text):
    
    ###Tokenizes text, removes stop words and punctuation, and lemmatizes tokens.
    
    tokens = parser(text)
    tokens = [token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_ for token in tokens]
    tokens = [token for token in tokens if token not in STOP_WORDS ]
    return tokens"""

# Preprocess texts
preprocessed_texts = [preprocess(text) for text in texts]

# Count vectorizer to convert preprocessed texts to a document-term matrix
vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
doc_term_matrix = vectorizer.fit_transform(preprocessed_texts)

# LDA model
lda_model = LatentDirichletAllocation(n_components=2, random_state=42)
lda_model.fit(doc_term_matrix)


# Assign topics to texts
for idx, text in enumerate(preprocessed_texts):
    topic = lda_model.transform(vectorizer.transform([text]))
    print(f"Text {idx+1} is about topic {topic.argmax()+1}")


Text 1 is about topic 2
Text 2 is about topic 2
Text 3 is about topic 2
Text 4 is about topic 2
Text 5 is about topic 2


/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
preprocessed_texts

['liver cancer rapidly increasing cancer among men woman united state estimated resulted death',
 'hepatocellular carcinoma hcc account primary liver cancer global incidence hcc expected increase million',
 'early detection hcc shown improve reception curative therapy overall survival',
 'several study shown mirna expression associated cancer patient survival',
 'saliva attractive biospecimen biomarkers noninvasive collected without requiring patient go doctor ’ office']

In [ ]:
dataset = [d.split() for d in preprocessed_texts]
dataset

In [ ]:

dictionary = gensim.corpora.Dictionary(dataset)
bow_corpus = [dictionary.doc2bow(doc) for doc in dataset]


In [ ]:

lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 2, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.054*"patient" + 0.032*"survival" + 0.032*"shown" + 0.032*"office" + 0.032*"collected" + 0.032*"biospecimen" + 0.032*"doctor" + 0.032*"’" + 0.032*"saliva" + 0.032*"requiring"
Topic: 1 
Words: 0.058*"cancer" + 0.058*"hcc" + 0.041*"liver" + 0.025*"million" + 0.025*"increase" + 0.025*"primary" + 0.025*"account" + 0.025*"incidence" + 0.025*"expected" + 0.025*"hepatocellular"


### LSA

In [ ]:
import numpy as np
import pandas as pd
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

#categories = ['comp.graphics', 'rec.autos', 'talk.politics.misc', 'sci.space', 'rec.sport.baseball']
#newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))


In [ ]:
print(newsgroups_train.data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
test1=[endemetriose,infertility]
preprocessed_test1 = []
for doc in test1:
    preprocessed_test1.append(preprocess(doc))

test2=[endemetriose,justice]
preprocessed_test2 = []
for doc in test2:
    preprocessed_test2.append(preprocess(doc))

test3=[endemetriose,bascketball]
preprocessed_test3 = []
for doc in test3:
    preprocessed_test3.append(preprocess(doc))

In [ ]:

preprocessed_test2

In [ ]:


lsa = TruncatedSVD(n_components=2, random_state=0)



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object to get the feature names
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
count_matrix = tfidf_vectorizer.fit_transform(preprocessed_test1)
lsa_matrix = lsa.fit_transform(count_matrix)
terms = tfidf_vectorizer.get_feature_names_out()

for i, topic in enumerate(lsa.components_):
    print(f"Top terms for topic #{i+1}:")
    print([terms[ind] for ind in topic.argsort()[:-6:-1]])


Top terms for topic #1:
['endometriosis', 'woman', 'uterus', 'pain', 'cause']
Top terms for topic #2:
['endometriosis', 'pain', 'cell', 'disease', 'outside']


In [ ]:
count_matrix = tfidf_vectorizer.fit_transform(preprocessed_test2)
lsa_matrix = lsa.fit_transform(count_matrix)
terms = tfidf_vectorizer.get_feature_names_out()

for i, topic in enumerate(lsa.components_):
    print(f"Top terms for topic #{i+1}:")
    print([terms[ind] for ind in topic.argsort()[:-6:-1]])

Top terms for topic #1:
['endometriosis', 'justice', 'consequence', 'pain', 'period']
Top terms for topic #2:
['endometriosis', 'pain', 'symptom', 'period', 'cell']


In [ ]:
count_matrix = tfidf_vectorizer.fit_transform(preprocessed_test3)
lsa_matrix = lsa.fit_transform(count_matrix)
terms = tfidf_vectorizer.get_feature_names_out()

for i, topic in enumerate(lsa.components_):
    print(f"Top terms for topic #{i+1}:")
    print([terms[ind] for ind in topic.argsort()[:-6:-1]])

Top terms for topic #1:
['endometriosis', 'basketball', 'player', 'pain', 'team']
Top terms for topic #2:
['endometriosis', 'pain', 'symptom', 'uterus', 'cell']


### Supervised method using a training dataset and LSTM model


In [ ]:
# import the dataset
data = pd.read_csv('/content/mtsamples.csv',index_col=0)
data.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
# Clean the dataset
filtered_data = data[['transcription', 'medical_specialty']]
filtered_data.loc[:, 'medical_specialty'] = filtered_data['medical_specialty'].apply(lambda x:str.strip(x))
mask = (filtered_data['medical_specialty'] == 'SOAP / Chart / Progress Notes') | \
       (filtered_data['medical_specialty'] == 'Office Notes') | \
       (filtered_data['medical_specialty'] == 'Consult - History and Phy.') | \
       (filtered_data['medical_specialty'] == 'Emergency Room Reports') | \
       (filtered_data['medical_specialty'] == 'Discharge Summary') | \
       (filtered_data['medical_specialty'] == 'Letters')
filtered_data = filtered_data[~mask]
data_categories  = filtered_data.groupby(filtered_data['medical_specialty'])
filtered_data_categories = data_categories.filter(lambda x:x.shape[0] > 100)
filtered_data_categories['medical_specialty'].value_counts()

<ipython-input-11-42ab1a664fef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'medical_specialty'] = filtered_data['medical_specialty'].apply(lambda x:str.strip(x))


Surgery                       1103
Cardiovascular / Pulmonary     372
Orthopedic                     355
Radiology                      273
General Medicine               259
Gastroenterology               230
Neurology                      223
Obstetrics / Gynecology        160
Urology                        158
Name: medical_specialty, dtype: int64

In [ ]:
data = filtered_data_categories.sample(frac=1.0)
df = data.dropna(subset=['transcription'])

In [ ]:
#Preproccess the 'Transcriptions' Before training
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def clean_text(text):
    # get English stopwords
    english_stopwords = set(stopwords.words('english'))
    
    # change to lower case and remove punctuation
    text = text.lower()
    
    # divide string into individual words
    tokens = word_tokenize(text)
    
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        tok = tok.strip() # remove space
        if tok not in english_stopwords:
            clean_tok = lemmatizer.lemmatize(tok) # lemmatizition
            clean_tok = stemmer.stem(clean_tok) # Stemming
            clean_tokens.append(clean_tok)
    return " ".join(clean_tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df['clean_trans']=df['transcription'].apply(clean_text)


<ipython-input-14-52ae22b041b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_trans']=df['transcription'].apply(clean_text)


In [ ]:
# Feature Engenniring for the specialty:X and Transcriptions:Y
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import numpy as np

MAX_TOKENS=3000
def indecies_vectorize(X_feature, y_label, max_length = 1500, max_tokens = MAX_TOKENS):
    '''
    X_feature: df['transcription'].values
    max_length: maximum number of words in one text/document
    max_tokens: the length of vocabulary
    '''
    
    le = LabelEncoder() 
    y_int = le.fit_transform(y_label) # we have 9 categories
    
    text_vectorization = layers.TextVectorization(
        max_tokens=max_tokens,
        output_mode="int",
        output_sequence_length=max_length,
    )
    # training the text vectorization object
    text_vectorization.adapt(X_feature) 
    # convert preprocessed training text into anrray of integers/indecies
    indecies_array = text_vectorization(X_feature) 
    
    # the data need some balancing
    smote_over_sample = SMOTE(sampling_strategy='minority')
    X, y = smote_over_sample.fit_resample(indecies_array, y_int)
    
    X = np.array(X)
    y = np.array(y)
    return X, y

In [ ]:
df['medical_specialty'].unique()

array(['Surgery', 'Neurology', 'Cardiovascular / Pulmonary', 'Orthopedic',
       'Radiology', 'Urology', 'Obstetrics / Gynecology',
       'General Medicine', 'Gastroenterology'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split
X_feature = df['clean_trans'].values
y_label = df['medical_specialty'].values

X, y = indecies_vectorize(X_feature, y_label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3027, 1500), (1010, 1500), (3027,), (1010,))

array([500, 457,   1, ...,   0,   0,   0])

In [ ]:
## Architect our model
import keras
def create_lstm_model(embedding_layer = None):
    # create input layer
    inputs = keras.Input(shape=(None,), dtype="int64")
    # add word embedding layer
    if embedding_layer is not None:
        embedded = embedding_layer(inputs)
    else:
        embedded = layers.Embedding(input_dim=MAX_TOKENS, output_dim=256, mask_zero=True)(inputs)
    # add LSTM layer
    x = layers.Bidirectional(layers.LSTM(32))(embedded)
    # add dropout layer
    x = layers.Dropout(0.3)(x)
    # add output layer
    outputs = layers.Dense(9, activation="softmax")(x)
    # combine all layers into one model
    lstm_model = keras.Model(inputs, outputs)
    # specifiy optimizer, loss, and metrics for the model
    lstm_model.compile(optimizer="rmsprop",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    # print the summay of the model architecture
    lstm_model.summary()
    
    return lstm_model

lstm_model = create_lstm_model()
    
    # define callback function
callbacks = [
        keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                        save_best_only=False)
    ]
# train model
lstm_model.fit(X_train, y_train, epochs=10, callbacks=callbacks)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         768000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               73984     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 9)                 585       
                                                                 
Total params: 842,569
Trainable params: 842,569
Non-trainable params: 0
_______________________________________________________

In [ ]:
!pip install h5py
lstm_model.save('lstm_model.h5')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
text_vectorization = layers.TextVectorization(
        max_tokens=MAX_TOKENS,
        output_mode="int",
        output_sequence_length=1500,
    )
    # training the text vectorization object
text_vectorization.adapt(X_feature) 

def preproccess_samples(text,vectorizer):
  #clean_text=cleaning(text)
  vector_text=vectorizer(text)
  preprocced_text=np.array(vector_text)
  return preprocced_text

In [ ]:
from keras.models import load_model
preprocced_intro=preproccess_samples(text=clean_text(intro),vectorizer=text_vectorization)
preprocced_intro=np.expand_dims(preprocced_intro, axis=0)

# load model from single file
model1 = load_model('lstm_model.h5')
# make predictions
intro_predection=model1.predict(preprocced_intro,verbose=0)
print(intro_predection)

[[0.055213   0.03864567 0.16080232 0.36524066 0.00380548 0.1151702
  0.02413589 0.04166114 0.19532566]]


In [ ]:
# the result for intro is : 'Orthopedic'

##Named Entity Recognition


In [ ]:
#using pretrainned biblio with spacy
nlp = spacy.load("en_core_web_sm")  # load the pre-trained English model

text = """Today we're starting to open up access to Bard, our early experiment that lets you collaborate with generative AI. You can use Bard to boost your productivity, accelerate your ideas and fuel your curiosity. We're beginning with the U.S. + U.K. and expanding over time to more countries and languages. Learn more and sign up."""

doc = nlp(text)  # apply the NLP pipeline to the text

# iterate over the entities in the document and print them
for ent in doc.ents:
    print(ent.text, ent.label_)

###Using the "en_core_web_em" of spacy

In [ ]:
#for the introductin
import spacy
 
nlp = spacy.load("en_core_web_sm")
text = intro

doc = nlp(text)
 
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)
#flair

In [ ]:
#for abstraction
text = abstraction

doc = nlp(text)
 

displacy.render(doc, style="ent", jupyter=True)

In [ ]:
#for materials
text = materials

doc = nlp(text)
 

displacy.render(doc, style="ent", jupyter=True)

In [ ]:
text = discution

doc = nlp(text)
 

displacy.render(doc, style="ent", jupyter=True)

###Using flair

In [ ]:
!pip install flair

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-english")

# make example sentence
sentence = Sentence("George Washington went to Washington")

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)


TypeError: ignored

In [ ]:
#for introduction
# make example sentence
sentence = Sentence(intro)

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

Sentence[516]: "Liver cancer is the most rapidly increasing cancer among men and women in the United States and is estimated to have resulted in 31,780 deaths in 2019 (American Cancer Society, 2019). Hepatocellular carcinoma (HCC) accounts for 80% of all primary liver cancers, and the global incidence of HCC is expected to increase to 78 million by 2030 (Petrick et al., 2016) due to increasing nonalcoholic steatohepatitis (NASH), hepatitis C, and excessive alcohol consumption. Early detection of HCC has shown to improve reception of curative therapy and overall survival (Singal, Pillai & Tiro, 2014). However, current HCC serum biomarkers, such as alpha fetoprotein (AFP) lack prognostic and diagnostic value (39–65% sensitivity) (Daniele et al., 2004). Although resection is the treatment of choice for many patients, as many as 54% will recur within 2 years, and ~70% within 5 years (Forner et al., 2006; Tabrizian et al., 2015). As early detection is associated with improved prognosis and 

In [ ]:
#for abstraction
# make example sentence
sentence = Sentence(abstraction)

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

Sentence[359]: "Background: Improved detection of hepatocellular carcinoma (HCC) is needed, as current detection methods, such as alpha fetoprotein (AFP) and ultrasound, suffer from poor sensitivity. MicroRNAs (miRNAs) are small, non-coding RNAs that regulate many cellular functions and impact cancer development and progression. Notably, miRNAs are detectable in saliva and have shown potential as non-invasive biomarkers for a number of cancers including breast, oral, and lung cancers. Here, we present, to our knowledge, the first report of salivary miRNAs in HCC and compare these findings to patients with cirrhosis, a high-risk cohort for HCC. Methods: We performed small RNA sequencing in 20 patients with HCC and 19 with cirrhosis. Eleven patients with HCC had chronic liver disease, and analyses were performed with these samples combined and stratified by the presence of chronic liver disease. P values were adjusted for multiple comparisons using a false discovery rate (FDR) approach a

In [ ]:
#for discution
# make example sentence
sentence = Sentence(discution)

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

Sentence[568]: "Current approaches used to screen patients for HCC lack sensitivity, resulting in too many false negative diagnoses (Daniele et al., 2004; Ayuso et al., 2018). There is a significant need for improved screening tools for HCC that are also non-invasive and cost-effective. Making screening tools more widely accessible may result in detection of HCC prior to the onset of symptoms, which could improve patient outcomes (Singal, Pillai & Tiro, 2014). The relative stability of miRNA (Fabian, Sonenberg & Filipowicz, 2010), combined with the non-invasive nature of saliva collection, makes this an attractive option to address these needs. Liver cirrhosis is the primary risk factor for development of HCC, so distinguishing patients with HCC from this cohort of high-risk patients serves as a proof-of-principle for this potential screening approach (Fattovich et al., 2004). Salivary miRNAs have demonstrated biomarker potential for CLD and cancers, such as oral cancer, lung cancer, a

In [ ]:
#sur l'article de president de la france
# make example sentence
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load("flair/ner-french")

# make example sentence
sentence = Sentence(macron)

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

TypeError: ignored

In [ ]:
import json
 
with open('/content/Corona2.json',encoding='utf-8' ) as f:
    data = json.load(f)
training_data = {'classes' : ['MEDICINE', "MEDICALCONDITION", "PATHOGEN"], 'annotations' : []}

for example in data['examples']:
  temp_dict = {'text': example['content'], 'entities': []}
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)

print(training_data['annotations'][0])

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("/content/training_data.spacy") # save the docbin object

100%|██████████| 31/31 [00:00<00:00, 400.79it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [ ]:
!curl -o base_config.cfg https://gist.githubusercontent.com/vinothpandian/d821b2ffd47682aa436a831e7e3e333e/raw/c15dd08676ece5df4e181d02499952d88d062de8/base_config.cfg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1678  100  1678    0     0   4877      0 --:--:-- --:--:-- --:--:--  4877


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg


2023-03-24 15:43:43.102161: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 15:43:43.102308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 15:43:43.102334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./trai

In [ ]:
!python -m spacy train base_config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

2023-03-24 15:44:53.996920: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 15:44:53.997040: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 15:44:53.997060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-03-24 15:45:00,683] [INFO] Set up n

In [ ]:
best_nlp = spacy.load("/content/model-best") #load the best model
doc = best_nlp(intro)

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

extraction des mots et phrases important(pagerank+transformers) et datasets

In [ ]:
import gensim
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

# define the TextRank algorithm function
def textrank(text):
    # tokenize the text into individual sentences
    sentences = sent_tokenize(text)
    # create a gensim TextRank summarizer object
   
    # apply the summarizer to the sentences to get the most important ones
    summary = gensim.summarization.summarize(text)
    # return the summary
    return summary

# define the text preprocessing function
def preprocess(text):
    # tokenize the text into individual words
    words = word_tokenize(text)
    # remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]
    # join the remaining words back into a string
    text = ' '.join(words)
    # return the preprocessed text
    return text

# example text to extract important sentences from
text = "Machine learning is the study of computer algorithms that improve automatically through experience. It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in a variety of applications, such as email filtering, natural language processing, and computer vision. There are many different types of machine learning algorithms, including supervised learning, unsupervised learning, and reinforcement learning."

# preprocess the text
preprocessed_text = preprocess(text)

# extract the important sentences using the TextRank algorithm
important_sentences = textrank(preprocessed_text)

# map the important sentences back to the original text
original_sentences = []
for sentence in important_sentences:
    # find the original position of the sentence in the original text
    start_index = text.find(sentence)
    end_index = start_index + len(sentence)
    # extract the sentence from the original text
    original_sentence = text[start_index:end_index]
    original_sentences.append(original_sentence)

# print the important sentences in the original form
for sentence in original_sentences:
    print(sentence)

In [ ]:
for sentence in important_sentences:
    print(sentence)

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install transformers==4.6.1
!pip install git+https://github.com/nlpyang/PreSumm.git

from transformers import BertTokenizer
from transformers import BertAbsSummarizer

# initialize tokenizer and summarizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
summarizer = BertAbsSummarizer.from_pretrained('bertabs-small-finetune-cnn')

# example text
text = """Basketball is a team sport in which two teams, most commonly of five players each, opposing one another on a rectangular court, compete with the primary objective of shooting a basketball (approximately 9.4 inches (24 cm) in diameter) through the defender's hoop (a basket 18 inches (46 cm) in diameter mounted 10 feet (3.048 m) high to a backboard at each end of the court), while preventing the opposing team from shooting through their own hoop. A field goal is worth two points, unless made from behind the three-point line, when it is worth three. After a foul, timed play stops and the player fouled or designated to shoot a technical foul is given one, two or three one-point free throws. The team with the most points at the end of the game wins, but if regulation play expires with the score tied, an additional period of play (overtime) is mandated.

Players advance the ball by bouncing it while walking or running (dribbling) or by passing it to a teammate, both of which require considerable skill. On offense, players may use a variety of shots – the layup, the jump shot, or a dunk; on defense, they may steal the ball from a dribbler, intercept passes, or block shots; either offense or defense may collect a rebound, that is, a missed shot that bounces from rim or backboard. It is a violation to lift or drag one's pivot foot without dribbling the ball, to carry it, or to hold the ball with both hands then resume dribbling.

The five players on each side fall into five playing positions. The tallest player is usually the center, the second-tallest and strongest is the power forward, a slightly shorter but more agile player is the small forward, and the shortest players or the best ball handlers are the shooting guard and the point guard, who implements the coach's game plan by managing the execution of offensive and defensive plays (player positioning). Informally, players may play three-on-three, two-on-two, and one-on-one.

Invented in 1891 by Canadian-American gym teacher James Naismith in Springfield, Massachusetts, in the United States, basketball has evolved to become one of the world's most popular and widely viewed sports.[1][2] The National Basketball Association (NBA) is the most significant professional basketball league in the world in terms of popularity, salaries, talent, and level of competition[3][4] (drawing most of its talent from U.S. college basketball). Outside North America, the top clubs from national leagues qualify to continental championships such as the EuroLeague and the Basketball Champions League Americas. The FIBA Basketball World Cup and Men's Olympic Basketball Tournament are the major international events of the sport and attract top national teams from around the world. Each continent hosts regional competitions for national teams, like EuroBasket and FIBA AmeriCup.

The FIBA Women's Basketball World Cup and Women's Olympic Basketball Tournament feature top national teams from continental championships. The main North American league is the WNBA (NCAA Women s Division I Basketball Championship is also popular), whereas the strongest European clubs participate in the EuroLeague Women."""

# tokenize the text
inputs = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=512, truncation=True)

# summarize the text
summary_ids = summarizer(inputs['input_ids'], num_beams=4, length_penalty=2.0, max_length=150, min_length=40, no_repeat_ngram_size=3)[0]
summary = tokenizer.decode(summary_ids, skip_special_tokens=True)

# print the summary
print(summary)


In [ ]:
!pip install yake

In [ ]:
import yake

# Specify language and max n-grams for keyword extraction
language = "en"
max_ngram_size = 3
def keywords(text,language,max_ngram_size):
  kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size)

  n_keywords = 10
  keywords = kw_extractor.extract_keywords(text)
  
  # Extract just the keywords
  keywords_list = [keyword[0] for keyword in keywords]
  
  print(keywords_list)


In [ ]:
keywords(intro,language,2)

['HCC', 'American Cancer', 'Cancer Society', 'United', 'rapidly increasing', 'cancer', 'Liver cancer', 'increasing cancer', 'miRNA', 'NASH', 'survival', 'cancers', 'Eslam', 'Valenti', 'Romeo', 'Early detection', 'Early', 'Liver', 'miRNAs', 'increasing']


In [ ]:
keywords(endemetriose,language,3)

['Endometriosis', 'pain', 'Key facts', 'symptoms', 'uterus', 'cells', 'pelvis', 'tissue', 'periods', 'pelvic', 'chronic pelvic pain', 'disease', 'endometriosis found', 'pelvic pain', 'chronic', 'people', 'body', 'found', 'severe pain', 'period']


In [ ]:
keywords(bascketball,language,3)

['Basketball World Cup', 'Olympic Basketball Tournament', 'Basketball', 'FIBA Women Basketball', 'Women Basketball World', 'rectangular court', 'defender hoop', 'diameter mounted', 'FIBA Basketball World', 'Women Olympic Basketball', 'Basketball World', 'National Basketball Association', 'primary objective', 'Olympic Basketball', 'Basketball Tournament', 'players', 'top national teams', 'national teams', 'opposing team', 'Basketball Champions League']


In [ ]:
keywords(infertility,language,3)

['fallopian tubes', 'woman', 'ovarian function', 'women', 'fallopian', 'ovarian', 'unprotected sex', 'uterus', 'function', 'infertility', 'tubes', 'PCOS', 'condition', 'POI', 'age', 'pituitary', 'menstrual cycle', 'Conditions', 'eggs', 'pregnant']


In [ ]:
!pip install keybert
!pip install keyphrase_vectorizers

In [ ]:
from keybert import KeyBERT

from keyphrase_vectorizers import KeyphraseCountVectorizer

vectorizer = KeyphraseCountVectorizer()
kw_model = KeyBERT()

In [3]:
kw_model.extract_keywords(docs=intro,vectorizer=KeyphraseCountVectorizer(),top_n=8)


NameError: ignored

In [ ]:
kw_model.extract_keywords(docs=endemetriose, vectorizer=vectorizer)


[('endometriosis', 0.7727),
 ('present endometriosis', 0.757),
 ('endometriosis symptoms', 0.6867),
 ('endometrioma', 0.6678),
 ('deep endometriosis', 0.6618)]

In [ ]:
kw_model.extract_keywords(docs=bascketball, vectorizer=vectorizer)


[('basketball', 0.5575),
 ('fiba basketball world cup', 0.4798),
 ('nba', 0.4583),
 ('hoop', 0.4535),
 ('defensive plays', 0.4488)]

In [8]:
kw_model.extract_keywords(docs=infertility, vectorizer=vectorizer,top_n=8)
#stop_words: Union[str, List[str]]='english',
#top_n: int=5,
#use_maxsum: bool=False, 
#use_mmr: bool=False, diversity: float=0.5, nr_candidates: int=20,
#vectorizer: CountVectorizer=None, 
#doc_embeddings: np.array=None, word_embeddings:

[('female infertility', 0.7136),
 ('infertility', 0.7006),
 ('fertility', 0.584),
 ('premature ovarian insufficiency', 0.5035),
 ('reproductive endocrinologist', 0.4899),
 ('reproductive endocrinologists', 0.4825),
 ('fertility treatments', 0.4819),
 ('unprotected sex', 0.4587)]

In [ ]:
kw_model.extract_keywords(docs=materials, vectorizer=vectorizer)


[('liver biopsy', 0.5423),
 ('chronic liver disease', 0.535),
 ('liver disease', 0.5217),
 ('liver transplantation', 0.4815),
 ('liver tumors', 0.4389)]

In [ ]:
kw_model.extract_keywords(docs=discution, vectorizer=vectorizer)


[('salivary mirnas', 0.4634),
 ('hcc lack sensitivity', 0.4559),
 ('hcc patients', 0.4512),
 ('hcc', 0.4026),
 ('improved screening tools', 0.4025)]

In [ ]:
!pip install pytextrank

In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy.lang.en import stop_words
from collections import Counter
import en_core_web_sm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import networkx as nx

# Load the spacy English language model
nlp = en_core_web_sm.load()

# Function to preprocess the text
def preprocess_text(text):
    # Create a spacy document object
    doc = nlp(text.lower())
    # Filter out stop words and punctuation and lemmatize the remaining words
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to extract important sentences using TextRank
def textrank_summary(text, num_sentences):
    # Preprocess the text
    preprocessed_text = preprocess_text(text)
    # Create a TfidfVectorizer object and fit it on the preprocessed text
    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform([preprocessed_text])
    # Get the feature names (i.e., the important words)
    feature_names = vectorizer.get_feature_names_out()
    # Create an empty matrix to hold the similarity scores between sentences
    similarity_matrix = np.zeros((len(feature_names), len(feature_names)))
    # Loop through each sentence and calculate its similarity to all other sentences
    for sentence in nlp(text).sents:
        # Preprocess the sentence and get its feature weights
        sentence_tokens = preprocess_text(str(sentence)).split()
        sentence_weights = []
        for feature in feature_names:
            if feature in sentence_tokens:
                sentence_weights.append(vectorizer.idf_[vectorizer.vocabulary_[feature]])
            else:
                sentence_weights.append(0)
        # Calculate the similarity between the sentence and all other sentences
        for i in range(len(feature_names)):
            for j in range(len(feature_names)):
                if i != j:
                    similarity_matrix[i][j] += (sentence_weights[i] * sentence_weights[j])
    # Convert the similarity matrix to a graph and run the PageRank algorithm on it
    similarity_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(similarity_graph)
    # Sort the sentences by their PageRank scores and return the top num_sentences sentences
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(nlp(text).sents)), reverse=True)
    summary = ""
    for i in range(num_sentences):
        summary += str(ranked_sentences[i][1]) + " "
    return summary

# Example usage



In [5]:
textrank_summary(infertility,3).split(".")

['The hypothalamus and pituitary glands in the brain produce hormones that maintain normal ovarian function',
 ' Chromopertubation is similar to a hysterosalpingogram but is done in the operating room at the time of a laparoscopy',
 ' Some of these conditions are listed below and can be evaluated using several different tests',
 '\n\n ']

In [11]:
textrank_summary(endemetriose,3).split(".")

['At present endometriosis is thought to arise due to:\n\nRetrograde menstruation is when menstrual blood containing endometrial cells flows back through the fallopian tubes and into the pelvic cavity at the time that blood is flowing out of the body through the cervix and vagina during periods',
 ' Some people also experience:\n\nchronic pelvic pain\nheavy bleeding during periods or between periods\ntrouble getting pregnant\nbloating or nausea\nfatigue\ndepression or anxiety',
 '\n Causes\nEndometriosis is a complex disease that affects many women globally from the onset of their first period (menarche) through menopause, regardless of ethnic origin or social status',
 ' ']

In [12]:
textrank_summary(intro,3).split(".")

['Although the majority of\nmiRNA regulation occurs intracellularly, in some cases select miRNAs are found in the\ncirculation and have been associated with a plethora of human diseases',
 ' However, current\nHCC serum biomarkers, such as alpha fetoprotein (AFP) lack prognostic and diagnostic\nvalue (39–65% sensitivity) (Daniele et al',
 ', 2004)',
 ' Early detection of HCC has shown to improve reception of\ncurative therapy and overall survival (Singal, Pillai & Tiro, 2014)',
 ' ']